# Belgian pharmacies

In [1]:
import json
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon
from ipywidgets import HTML
stats = json.load(open("stats.json"))
pharmacies = json.load(open("last-pharmacies.json"))

## Map of pharmacies

In [6]:
center = (50.8503396, 4.3517103) # Brussels
m = Map(center=center)

# Icons
icon_active = AwesomeIcon(name='house-chimney-medical', marker_color='blue', icon_color='darkgreen')
icon_closed = AwesomeIcon(name='house-medical-circle-xmark', marker_color='red', icon_color='darkgreen')

# Create markers
markers = []
for pharmacy in pharmacies:
    icon_state = icon_active if pharmacy["status"].casefold() == "active" else icon_closed
    # TODO later change that trade-off
    geo = pharmacy["geo"][1]
    coords = (geo["latitude"], geo["longitude"])
    name = pharmacy["name"]
    address = pharmacy["textual_address"]
    zipCode = pharmacy["zipCode"]
    municipality = pharmacy["municipality"]
    # Text when clicked on marker
    message = HTML()
    message.value = "<b>{}</b><br>{}<br>{} {}".format(name, address, zipCode, municipality)
    message.placeholder = ""
    message.description = ""
    marker = Marker(location=coords,icon=icon_state, title=name, alt=name, draggable=False)
    marker.popup = message
    markers.append(marker)

marker_cluster = MarkerCluster(markers=markers)
m.add_layer(marker_cluster)

m

Map(center=[50.8503396, 4.3517103], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…